#### CS Prediction Algorithm

In [1]:
import pandas as pd
import numpy as np
import pathlib
from pathlib import Path
import itertools
from itertools import product, combinations
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from wwknn import WWKNeighborsClassifier
import joblib
from joblib import dump,load


Neucube File Prep

In [ ]:
Directory = r'C:\Users\Lab Admin\Documents\NIC'

pathlist = Path(Directory).rglob('*.easy')



Get 2 seconds of data

In [2]:
#Determines directory and pathlist

Directory = r'C:\Users\Lab Admin\Desktop\ML test' #as needed

pathlist = Path(Directory).rglob('*.csv')


for path in pathlist:
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    df = pd.read_csv(path, header = None)
    x= df.values #gets a dataframe
    x1 = x[30*500:] #starts data from the starting row at 30 seconds
    x2 = x1[:2*500] #selects data until 2s mark
    x3 = x2[:,~np.all(np.isnan(x2), axis=0)]
    np.savetxt(filename+'cleaned'+'.csv', x3, fmt = '%s', delimiter = ',') #saves array to csv of the EEG data (30-32s)
    np.savetxt('sam1'+'.csv', x3, fmt = '%s', delimiter = ',') #saves array to csv of the EEG data (30-32s)
    np.savetxt('sam2'+'.csv', x3, fmt = '%s', delimiter = ',') #saves array to csv of the EEG data (30-32s)

Now feed to neucube

Now open the 1503 outlayer weight file, delete the row with all -5 and run the next code

#### Prediction

In [4]:
#Determines directory and pathlist

Directory = r'C:\Users\Lab Admin\Desktop\ML test' #as needed

pathlist = Path(Directory).rglob('*.csv')


for path in pathlist:
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    df = pd.read_csv(path, header = None)
    N = 1471
    df = df.iloc[: , N:]   # Drop first N columns (SNNr Neurons) of dataframe 
    df.columns = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 
    'Fp2', 'Fp1', 'AF3', 'Fz', 'FC2', 'Cz', 'CP2', 'PO3', 'O1', 'Oz', 'O2', 
    'PO4', 'Pz', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']  #sets column names
    df1 = (df[['P4', 'Fz', 'Cz']]) #chooses specific columns (5 eeg channels for neucube training)
    x= df1.values #gets a dataframe
    print(x)
    CSprediction = load('CS_prediction.pkl') #loads prediction model
    compute = CSprediction.predict(x) #predicts based on sample data
    print(compute)
    #np.savetxt('predict.csv', x, fmt = '%s', delimiter = ',') #saves array to csv of the EEG data (90-92s)
    #df1.to_csv('sam65.csv', index=False, header=None) #saves dataframe to csv named sam65
    
# transfer output to neucube for processing

[[-2.815  -2.3638 -2.74  ]
 [-2.815  -2.3638 -2.74  ]]
[0 0]


#### ECG data cleaner

In [ ]:
Directory = '/Users/yaale244/Desktop/Machine learning workspace/working files 2' #change directory as needed

pathlist = Path(Directory).rglob('*.csv')
sortedfiles = []
for sf in sorted(pathlist):
    sortedfiles.append(sf) 
for path in sortedfiles:
    path_in_str = str(path) # because path is object not string
    filename = pathlib.Path(path).name #finds the file name
    df = pd.read_csv(path_in_str, header = None) #reads the file
    x= df.values #gets a dataframe
    x1 = x[75*512:] #starts data from the starting row at 75 seconds
    x2 = x1[:30*512] #selects data until 30s mark
    x3 = (x2[:, ~np.isnan(x2).any(axis=0)]) #removes any Nans
    np.savetxt('ECG kubios rdy.txt', x3, fmt = '%s', delimiter = ',') #saves a csv of the cleaned 30s ECG data (75-105s)

#transfer output to kubios to get SNS index

#### Train/retrain prediction model (outlayer weights + SNS)

In [ ]:
#Fuse outlayerweights from neucube with SNS index

Directory = r"C:\Users\Lab Admin\Desktop\ML test"
pathlist = Path(Directory).rglob('*.csv')

#reads class labels, code can now recognize the file
y = pd.read_csv("tar_class_labels.csv", header=None).values.flatten()
le = LabelEncoder()
y = le.fit_transform(y)
y

#determines parameters to iterate over

parameters = {"n_neighbors" : [ 7 ],
 "neighbors_type"  : [ 'per_class' ],
     "p"           : [ 2 ],
 "initial_weights" : [ 'snr' ],
 "voting_type"    : [ 'inverse' ],
 "voting_weights" : [ 'snr' ],
 "n_features_to_select" : [None] }

param_grid = list(product(*parameters.values())) 

for path in pathlist:
    path_in_str = str(path)# because path is object not string
    filename = pathlib.Path(path_in_str).name
    df = pd.read_csv(path, header = None)
        
    results = []
    X = df.values 

    for grid in param_grid:
        loo = LeaveOneOut()
        loo.get_n_splits(X)
        predictions = []
        itr = 0       
        
        for train_index, test_index in loo.split(X):
            
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
                
            m_wwknn = WWKNeighborsClassifier(n_neighbors=grid[0], neighbors_type=grid[1], p=grid[2], initial_weights=grid[3], voting_type=grid[4], voting_weights=grid[5]).fit(X_train,y_train) #sets parameters for classifer to run through
            y_pred_wwknn = int(m_wwknn.predict(X_test)) #predicts based on X_test, added int so that predictions can be visualised in confusion matrix
            predictions.append(y_pred_wwknn) # appends predicitions for every sample to a list


        #if you put anything in this indented line, it will equate it to the last output in the for loop it aligns with, in this case its for every grid combo

        percentage = (accuracy_score(y,predictions)) #creates overall accuracy score based on predictions for a set of parameters
        done = (np.concatenate((percentage, grid, filename), axis = None)) # concatenates accuracy, parameters, channels , filename
        results.append(done) #appends this current combination of parameters and channels to the results.

print(results)
y_true = y
print(predictions)
joblib.dump(m_wwknn, 'testCS_prediction.pkl', compress=9)